<a href="https://colab.research.google.com/github/Mariammmmmmm/modeling_labs/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%962_%D0%BF%D0%BE_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 64 kB/s 
     |████████████████████████████████| 199 kB 62.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=17f505707f9c2edd0de5832d0c51f5597ca8f456ce32a2c31713741a5d547803
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Dataframe').getOrCreate()
spark

In [35]:
part0 = spark.read.parquet('part_0.parquet')
part0 = part0.dropna()
part1 = spark.read.parquet('part_1.parquet')
part1 = part1.dropna()
part2 = spark.read.parquet('part_2.parquet')
part2 = part2.dropna()
part3 = spark.read.parquet('part_3.parquet')
part3 = part3.dropna()
part4 = spark.read.parquet('part_4.parquet')
part4 = part4.dropna()
part5 = spark.read.parquet('part_5.parquet')
part5 = part5.dropna()
part0 = part0.union(part1).union(part2)
part0 = part0.dropna()
part0_test = part3.union(part4).union(part5)
part0.show(5)

+------+-------------------+--------+--------------+---------+--------------+--------------------+--------------+--------------+-----------+---+-------+----+------------+-----------+----+-----------+----------+---------+------------------+-----------------+
|app_id|               amnt|currency|operation_kind|card_type|operation_type|operation_type_group|ecommerce_flag|payment_system|income_flag|mcc|country|city|mcc_category|day_of_week|hour|days_before|weekofyear|hour_diff|transaction_number|__index_level_0__|
+------+-------------------+--------+--------------+---------+--------------+--------------------+--------------+--------------+-----------+---+-------+----+------------+-----------+----+-----------+----------+---------+------------------+-----------------+
|     0| 0.4654254330729043|       1|             4|       98|             4|                   2|             3|             7|          3|  2|      1|  37|           2|          4|  19|        351|        34|       -1|      

In [36]:
part0_test.show(5)

+------+-------------------+--------+--------------+---------+--------------+--------------------+--------------+--------------+-----------+---+-------+----+------------+-----------+----+-----------+----------+---------+------------------+-----------------+
|app_id|               amnt|currency|operation_kind|card_type|operation_type|operation_type_group|ecommerce_flag|payment_system|income_flag|mcc|country|city|mcc_category|day_of_week|hour|days_before|weekofyear|hour_diff|transaction_number|__index_level_0__|
+------+-------------------+--------+--------------+---------+--------------+--------------------+--------------+--------------+-----------+---+-------+----+------------+-----------+----+-----------+----------+---------+------------------+-----------------+
| 70093| 0.3349274118365794|       1|             1|       88|             2|                   1|             1|             2|          1|  1|      1|  73|           1|          2|  16|        357|        43|       -1|      

In [37]:
part0.count()

16227169

In [38]:
train_target = spark.read.csv('train_target.csv', header=True)
train_target = train_target.dropna()
train_target.show(5)

+------+-------+----+
|app_id|product|flag|
+------+-------+----+
|     0|      3|   0|
|     1|      1|   0|
|     2|      1|   0|
|     3|      1|   0|
|     4|      1|   0|
+------+-------+----+
only showing top 5 rows



In [39]:
splitweights = [0.70,0.30]
seed = 10
traindf, testdf = train_target.randomSplit(splitweights, seed)
traindf.count()

674062

In [40]:
testdf.count()

289749

In [41]:
from pyspark.sql.types import IntegerType
part0 = part0.join(traindf, (part0.app_id == traindf.app_id))
part0 = part0.withColumn("flag", part0["flag"].cast(IntegerType()))
part0.show(5)

+------+-------------------+--------+--------------+---------+--------------+--------------------+--------------+--------------+-----------+---+-------+----+------------+-----------+----+-----------+----------+---------+------------------+-----------------+------+-------+----+
|app_id|               amnt|currency|operation_kind|card_type|operation_type|operation_type_group|ecommerce_flag|payment_system|income_flag|mcc|country|city|mcc_category|day_of_week|hour|days_before|weekofyear|hour_diff|transaction_number|__index_level_0__|app_id|product|flag|
+------+-------------------+--------+--------------+---------+--------------+--------------------+--------------+--------------+-----------+---+-------+----+------------+-----------+----+-----------+----------+---------+------------------+-----------------+------+-------+----+
|     0| 0.4654254330729043|       1|             4|       98|             4|                   2|             3|             7|          3|  2|      1|  37|         

In [42]:
part0_test = part0_test.join(testdf, (part0_test.app_id == testdf.app_id))
part0_test = part0_test.withColumn("flag", part0_test["flag"].cast(IntegerType()))
part0_test.show(5)

+------+-------------------+--------+--------------+---------+--------------+--------------------+--------------+--------------+-----------+---+-------+----+------------+-----------+----+-----------+----------+---------+------------------+-----------------+------+-------+----+
|app_id|               amnt|currency|operation_kind|card_type|operation_type|operation_type_group|ecommerce_flag|payment_system|income_flag|mcc|country|city|mcc_category|day_of_week|hour|days_before|weekofyear|hour_diff|transaction_number|__index_level_0__|app_id|product|flag|
+------+-------------------+--------+--------------+---------+--------------+--------------------+--------------+--------------+-----------+---+-------+----+------------+-----------+----+-----------+----------+---------+------------------+-----------------+------+-------+----+
| 70094|0.47905506364463324|       1|             4|      175|             4|                   2|             1|             3|          2|  2|      1|   4|         

In [43]:
part0_test.printSchema

<bound method DataFrame.printSchema of DataFrame[app_id: int, amnt: double, currency: int, operation_kind: int, card_type: int, operation_type: int, operation_type_group: int, ecommerce_flag: int, payment_system: int, income_flag: int, mcc: int, country: int, city: int, mcc_category: int, day_of_week: int, hour: int, days_before: int, weekofyear: int, hour_diff: bigint, transaction_number: int, __index_level_0__: bigint, app_id: string, product: string, flag: int]>

In [44]:
cols = list(set(part0.columns))
cols

['operation_kind',
 'city',
 'day_of_week',
 'weekofyear',
 'currency',
 'days_before',
 'hour_diff',
 'operation_type',
 'payment_system',
 'app_id',
 'amnt',
 'product',
 'flag',
 'transaction_number',
 'hour',
 'mcc',
 'country',
 'income_flag',
 '__index_level_0__',
 'operation_type_group',
 'ecommerce_flag',
 'card_type',
 'mcc_category']

In [45]:
print(set(part0.columns) - set('flag'))

{'operation_kind', 'city', 'flag', 'day_of_week', 'weekofyear', 'hour', 'currency', 'days_before', 'hour_diff', 'operation_type', 'mcc', 'payment_system', 'country', 'operation_type_group', 'app_id', 'income_flag', '__index_level_0__', 'amnt', 'transaction_number', 'ecommerce_flag', 'card_type', 'mcc_category', 'product'}


In [46]:
len(cols)

23

In [86]:
arr = ['hour_diff',
 'mcc',
 'amnt',
 'mcc_category',
 'days_before',
 'country',
 'weekofyear',
 'operation_kind',
 'city',
 '__index_level_0__',
 'hour',
 'income_flag',
 'card_type',
 'currency',
 'day_of_week',
 'payment_system',
 'ecommerce_flag',
 'operation_type_group',
 'transaction_number',
 'operation_type']

In [87]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=arr, outputCol='features')
output = assembler.transform(part0)
df_final_train = output.select('features', 'flag')
df_final_train.show(5)

+--------------------+----+
|            features|flag|
+--------------------+----+
|[-1.0,2.0,0.46542...|   0|
|[0.0,2.0,0.0,2.0,...|   0|
|[0.0,2.0,0.521151...|   0|
|[52.0,10.0,0.3560...|   0|
|[280.0,2.0,0.0,2....|   0|
+--------------------+----+
only showing top 5 rows



In [88]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=arr, outputCol='features')
output = assembler.transform(part0)
df_final_test = output.select('features', 'flag')
df_final_test.show(5)

+--------------------+----+
|            features|flag|
+--------------------+----+
|[-1.0,2.0,0.46542...|   0|
|[0.0,2.0,0.0,2.0,...|   0|
|[0.0,2.0,0.521151...|   0|
|[52.0,10.0,0.3560...|   0|
|[280.0,2.0,0.0,2....|   0|
+--------------------+----+
only showing top 5 rows



In [89]:
df_final_train.printSchema

<bound method DataFrame.printSchema of DataFrame[features: vector, flag: int]>

In [90]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="flag")
lrm = lr.fit(df_final_train)
lrm

LogisticRegressionModel: uid=LogisticRegression_9e0237e210a9, numClasses=2, numFeatures=20

In [91]:
lrm_summary = lrm.summary
lrm_summary.predictions.show(5)

+--------------------+----+--------------------+--------------------+----------+
|            features|flag|       rawPrediction|         probability|prediction|
+--------------------+----+--------------------+--------------------+----------+
|[-1.0,2.0,0.46542...| 0.0|[3.20249699231992...|[0.96092813538995...|       0.0|
|[0.0,2.0,0.0,2.0,...| 0.0|[2.17327671238462...|[0.89782395146380...|       0.0|
|[0.0,2.0,0.521151...| 0.0|[3.27674000080246...|[0.96362217862364...|       0.0|
|[52.0,10.0,0.3560...| 0.0|[3.15113358590704...|[0.95895336501263...|       0.0|
|[280.0,2.0,0.0,2....| 0.0|[2.11993883246883...|[0.89282607667535...|       0.0|
+--------------------+----+--------------------+--------------------+----------+
only showing top 5 rows



In [92]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
pred_lables = lrm.evaluate(df_final_test)
result = pred_lables.predictions
result.show(5)

+--------------------+----+--------------------+--------------------+----------+
|            features|flag|       rawPrediction|         probability|prediction|
+--------------------+----+--------------------+--------------------+----------+
|[-1.0,2.0,0.46542...|   0|[3.20249699231992...|[0.96092813538995...|       0.0|
|[0.0,2.0,0.0,2.0,...|   0|[2.17327671238462...|[0.89782395146380...|       0.0|
|[0.0,2.0,0.521151...|   0|[3.27674000080246...|[0.96362217862364...|       0.0|
|[52.0,10.0,0.3560...|   0|[3.15113358590704...|[0.95895336501263...|       0.0|
|[280.0,2.0,0.0,2....|   0|[2.11993883246883...|[0.89282607667535...|       0.0|
+--------------------+----+--------------------+--------------------+----------+
only showing top 5 rows



In [93]:
from pyspark.ml.feature import VectorAssembler
vector_col = 'corr-feature'
assemblerCorr = VectorAssembler(inputCols=['hour_diff',
 'mcc',
 'amnt',
 'mcc_category',
 'days_before',
 'country',
 'weekofyear',
 'operation_kind',
 'city',
 '__index_level_0__',
 'hour',
 'income_flag',
 'card_type',
 'currency',
 'day_of_week',
 'payment_system',
 'ecommerce_flag',
 'operation_type_group',
 'transaction_number',
 'operation_type',
 'flag'], outputCol=vector_col)
df_vector = assemblerCorr.transform(part0).select(vector_col)
df_vector.show(5)

+--------------------+
|        corr-feature|
+--------------------+
|[-1.0,2.0,0.46542...|
|[0.0,2.0,0.0,2.0,...|
|[0.0,2.0,0.521151...|
|[52.0,10.0,0.3560...|
|[280.0,2.0,0.0,2....|
+--------------------+
only showing top 5 rows



In [94]:
from pyspark.ml.stat import Correlation
matrix = Correlation.corr(df_vector, vector_col).collect()[0][0]
corrmatrix = matrix.toArray().tolist()
print(corrmatrix)

[[1.0, -0.038453078338358745, 0.13079317983799266, -0.040554030027362506, 0.00038706562894230483, -0.023600546132377657, -0.008638571608199301, 0.2109280112625824, 0.004253295530725577, -0.004389931054546503, -0.037986809420990905, 0.2151561021803733, 0.01262240889037704, -0.017251780904331066, 0.001378093096700919, 0.10637670185780303, -0.01565932362704104, 0.20493898057539991, -0.15864621823260475, 0.09677488493941776, 0.006514607831885588], [-0.038453078338358745, 1.0, 0.03386459657447085, 0.6875192990381621, -0.0045488154556083986, 0.15909470906102435, 0.006694800975430345, -0.13307079015187462, 0.029375739760667825, -0.003033352189460087, -0.04706003310919703, -0.10575301340260174, 0.03938596454196586, 0.08468068359233424, 0.008184758102219698, -0.1152312896994495, 0.36105954906882654, -0.07412695127704191, 0.06010168652480338, 0.019069285986434346, -0.012732095021748841], [0.13079317983799266, 0.03386459657447085, 1.0, 0.05187254367682303, 0.023991986046865643, 0.0253558786929242

In [95]:
df_corr = spark.createDataFrame(corrmatrix, ['hour_diff',
 'mcc',
 'amnt',
 'mcc_category',
 'days_before',
 'country',
 'weekofyear',
 'operation_kind',
 'city',
 '__index_level_0__',
 'hour',
 'income_flag',
 'card_type',
 'currency',
 'day_of_week',
 'payment_system',
 'ecommerce_flag',
 'operation_type_group',
 'transaction_number',
 'operation_type'
 'flag'])
df_corr.show(25)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|           hour_diff|                 mcc|                amnt|        mcc_category|         days_before|             country|          weekofyear|      operation_kind|                city|   __index_level_0__|                hour|         income_flag|           card_type|            currency|         day_of_week|      payment_system|      ecommerce_flag|operation_type_group|  transaction_number|  operation_typeflag|                 _21|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------

**Наибольшая корреляция по времени, дне недели и ecommerce_flag.**

In [96]:
result.select(["flag", "prediction"]).show(5)

+----+----------+
|flag|prediction|
+----+----------+
|   0|       0.0|
|   0|       0.0|
|   0|       0.0|
|   0|       0.0|
|   0|       0.0|
+----+----------+
only showing top 5 rows



In [97]:
tp = result[(result.flag==1) & (result.prediction==1)].count()
tn = result[(result.flag==0) & (result.prediction==0)].count()
fp = result[(result.flag==0) & (result.prediction==1)].count()
fn = result[(result.flag==1) & (result.prediction==0)].count()

In [98]:
accuracy = float((tn+tp)/(result.count()))
accuracy

0.9787667573634494

In [99]:
recall = float(tn)/float(tp+tn)
recall

1.0

In [100]:
eval = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="flag")
auc = eval.evaluate(pred_lables.predictions)
auc

0.5